In [2]:
a=[0,1,2,3,4,5]
a[:5]

[0, 1, 2, 3, 4]

In [326]:
class Node:
    def __init__(self, isleaf=False):
        self.isleaf=isleaf
        self.keys=[]
        self.child=[]
    def __repr__(self):
        return f'Node :{self.keys}'
    def split(self):
        length = len(self.keys)
        temp = Node(self.isleaf)
        mid = self.keys[length//2]
        temp.keys = self.keys[length//2+1:]
        self.keys = self.keys[:length//2]
        if self.child:
            temp.child = self.child[length//2+1:]
            self.child = self.child[:length//2+1]
        return self,mid,temp
        
# nn=Node()
# nn.keys=[10,20,30,40,50]
# nn.child=[[1,2],[12,13],[22,23],[33,39],[45,49],[55,60]]
# ll,mid,rr=nn.split()
# ll,ll.child , rr, rr.child

In [477]:
class Node:
    def __init__(self, isleaf=False):
        self.isleaf=isleaf
        self.keys=[]
        self.child=[]
    def __repr__(self):
        return f'Node :{self.keys}'
    def split(self):
        length = len(self.keys)
        temp = Node(self.isleaf)
        mid = self.keys[length//2]
        temp.keys = self.keys[length//2+1:]
        self.keys = self.keys[:length//2]
        if self.child:
            temp.child = self.child[length//2+1:]
            self.child = self.child[:length//2+1]
        return self,mid,temp

class BTree:
    def __init__(self, m):
        self.m=m #차수 -> m개의 child를 가질 수 있음
        self.root=Node(True)
        # self.root.isleaf = 'root'
        # self.root=None

    # k : Key 
    # def insert_node(self,p_pos,p_node,node,key):
    def insert_node(self,node,key):
        """ 데이터 추가하는것, Root부터 시작해서 내려가도록 Trigger가 발동되며, 
        현재의 노드(node)가 leaf가 아닐경우, 재귀적으로 child를 내려감. 
        리프에서의 삽입이 이뤄지고나면, 그대로 자기자신만 반환하고, 
        Leaf에서의 삽입이 OverFlow가 일어날경우 중간key와 자기자신을 반환
        삽입을 하고난 이후, 반환되는 key가 있다면, 해당 child에서 OverFlow가 난것이므로, 관련연산 진행
        """
        pos=0
        while pos < len(node.keys):
            # print(pos)
            if key[0]==node.keys[pos][0]:
                #Duplicate
                print(f"Duplicates key {key}")
                return node, None 
            elif key[0]<node.keys[pos][0]: break
            pos+=1
        # print(key," Node : ",node , pos)
        
        # Position 찾기 완료
        if node.isleaf==True : # Leaf라면?? -> Leaf에서의 삽입 로직
            i=len(node.keys)-1
            node.keys.append([1e9,None])
            while i>=0 and key[0]<node.keys[i][0]:
                node.keys[i+1]=node.keys[i]
                i-=1
            node.keys[i+1]=key
            if len(node.keys) >= self.m : # 오버플로 발생?
                if node == self.root:
                    # print("Root OverFlow")
                    ll,mid,rr=node.split()
                    self.root=Node(False)
                    self.root.keys.append(mid)
                    self.root.child.append(ll)
                    self.root.child.append(rr)
                    return self.root, None
                else:
                    # print("Leaf OVERFLOW",node)
                    return node, node.keys[(self.m//2)]

            else: return node, None
        # elif node.isleaf == False: #Leaf가 아니라면?
        else: #Leaf가 아니라면?
            node.child[pos], child_mid_kv = self.insert_node(node.child[pos],key)
            if child_mid_kv: # Child에서 OverFlow발생 -> 자기에 추가시켜줘야해
                ll,mid,rr=node.child[pos].split()

                i=len(node.keys)-1
                node.keys.append([1e9,None])
                node.child.append([])
                while i>=0 and mid[0] < node.keys[i][0]:
                    node.keys[i+1]=node.keys[i]
                    node.child[i+2]=node.child[i+1]
                    i-=1
                node.keys[i+1]=mid
                # i=len(node.child)-1
                node.child[i+2]=rr
                if len(node.keys) >= self.m : # 오버플로 발생?
                    if node == self.root:
                        # print("Root OverFlow")
                        ll,mid,rr=node.split()
                        self.root=Node(False)
                        self.root.keys.append(mid)
                        self.root.child.append(ll)
                        self.root.child.append(rr)
                        return self.root, None
                if len(node.keys)>=self.m:
                    return node, node.keys[(self.m//2)]
            return node, None

    def search_key(self,node,key):
        """ Find Key 
        IF FIND : RETURN [Node, Key]
        else : RETURN [None, None] """
        # print(node)
        pos=0
        while pos < len(node.keys):
            if key[0] == node.keys[pos][0]:
                # print(f'Find! key : {node.keys[pos]}')
                return node, node.keys[pos]
            elif key[0] < node.keys[pos][0] : break
            else:
                pos+=1
        # print(node, pos)
        if node.child:
            return self.search_key(node.child[pos],key)
        else : 
            return None, None
            # return f"NOT FOUND {key}"

    def print_inorder(self,node):
        if node.child:
            for i in range(len(node.keys)):
                self.print_inorder(node.child[i])
                print(node.keys[i])
            self.print_inorder(node.child[-1])
        else:
            for i in node.keys:
                print(i)
    
    def find_change_Pred(self,node,key):
#특정키의 왼쪽자식받아 오른쪽을 찾아내려가서 가장큰수의 키를 바꾸고 바꾸기전 key return
        if node.isleaf: 
            temp = node.keys[-1]
            node.keys[-1] = key
            return temp
        return self.find_change_Pred(node.child[-1],key)

    def find_change_Succ(self,node,key):
#특정키의 오른쪽자식을 받아 왼쪽으로쭉내려가 가장 작은수의 키를 바꾸고 바꾸기전 key return
        if node.isleaf:
            temp=node.keys[0]
            node.keys[0] = key
            return temp
        return self.find_change_Succ(node.child[0],key)


## DELETE PACKER
    def delete_pack(self,node,key):
        self.delete(node,key)
        if self.root.keys:
            pass
        else:
            pass
            self.root = self.root.child[0]
            # self.root.child = [i for i in self.root.child]
            # self.root = [i for i in self.root.child]

## DELETE
    def delete(self,node,key):
        pos = 0
        # FIND POSITION
        while pos<len(node.keys) and key[0] > node.keys[pos][0]:
            pos+=1

        print("DELETE",node,"KEY : ",key,pos)

        if node.isleaf==True : # Leaf 에서 키를 찾든 못찾든
            if pos<len(node.keys) and key[0]==node.keys[pos][0]:
                node.keys.pop(pos)
                print("find node : ",node,' key : ',key)
                # return "DELETE FROM LEAF COMPLETED" # 삭제연산완료표시
                return node
            else :
                print("NOT FOUND")
                return node # "NOT FOUND" # 리프까지 갔는데 못찾음
        else: # Leaf가 아닌 경우
            if pos < len(node.keys):
                if key[0]==node.keys[pos][0]:
                    # 리프가 아닌데 찾은경우
                    ## Leaf에서 찾아서(Pred or Succ) 바꿔주는 연산
                    ## 바꾼이후 해당방향 child로 delete연산수행
                    print(f"find_Internal -> change, Now Node : {node}, Now Pos : {pos}")
                    # if len(node.child[pos].keys)>= self.m:
                    #     node.keys[pos] = self.find_change_Pred(node.child[pos],key)
                    # elif len(node.child[pos+1].keys)>= self.m:
                    #     node.keys[pos] = self.find_change_Succ(node.child[pos+1],key)
                    # else: ## Merge해야하는경우
                    #     self.merge()
                    #     pass
                    if len(node.child[pos].keys) >= len(node.child[pos].keys):
                        print('changePred (left)',pos)
                        node.keys[pos] = self.find_change_Pred(node.child[pos],key)
                        self.delete(node.child[pos], key)
                        self.balancing_child(node,pos,pos)
                    else:
                        print('changeSucc (right)',pos)
                        node.keys[pos] = self.find_change_Succ(node.child[pos+1],key)
                        self.delete(node.child[pos+1], key)
                        self.balancing_child(node,pos,pos+1)
                    # pass ## TODO : Internal Node Deletion Process
                else:
                    self.delete(node.child[pos],key)    
            else: # 리프가 아닌데 키값도 아닌데 pos는찾은경우 => 아래로 내려가자
                self.delete(node.child[pos],key)
        
        print(f"Check The Node {node} 's Stability")
        ## Check Child Keys for M
        if len(node.child[pos].keys)>= self.m//2:
            print("It is safe , RETURN",node)
            return node
        else:
            ## Not Enough Keys For child[pos] => Balancing
            print("Not Stable => Balancing")
            node = self.balancing_child(node,pos,pos)
            return node
        

    def balancing_child(self,pnode,pos,cpos):
        print("Balancing Child ", pnode ,"cpos :", cpos , pnode.child[cpos])
        if cpos==0:# 맨 왼쪽 자식
            print("맨 왼쪽 자식 rebalancing")
            if self.distribute_with_right(pnode,cpos):
                return pnode
            else:
                print("Merge")  
                self.merge(pnode,pos,cpos,cpos+1)
                pass
                ## Merge
        elif cpos==len(pnode.child)-1: # 맨 오른쪽 자식
            print("맨 오른쪽 자식 rebalancing")
            if self.distribute_with_left(pnode,cpos):
                return pnode
            else:
                print("Merge") 
                self.merge(pnode,pos-1,cpos-1,cpos)
                pass
            ## Merge
        else:
            print("중간에 껴있는 자식")
            if self.distribute_with_left(pnode,cpos) :
                return pnode
            elif self.distribute_with_right(pnode,cpos):
                return pnode
            else:
                print("중간녀석의 Merge")
                pass


    def distribute_with_right(self, pnode,pos):
        if len(pnode.child[pos+1].keys) >= self.m//2+1:# 오른쪽으로부터 가져오기 가능
            pnode.child[pos].keys.append(pnode.keys[pos])
            pnode.keys[pos] = pnode.child[pos+1].keys[0]
            pnode.child[pos+1].keys.pop(0)
            if pnode.child[pos+1].child: #자식이 있다면? 옮겨진곳으로 붙여주자구
                pnode.child[pos].child.append(pnode.child[pos+1].child[0])
                pnode.child[pos+1].child.pop(0)
            return True
        else:
            return False
    def distribute_with_left(self, pnode,pos):
        if len(pnode.child[pos-1].keys) >= self.m//2 + 1: # 왼쪽으로부터 가져오기 가능
            print(f"FROM LEFT pnode: {pnode} , pos: {pos} , child: {pnode.child}")
            pnode.child[pos].keys = [pnode.keys[pos-1]] + pnode.child[pos].keys
            pnode.keys[pos-1] = pnode.child[pos-1].keys[-1]
            pnode.child[pos-1].keys.pop()
            if pnode.child[pos-1].child:
                pnode.child[pos].child = [pnode.child[pos-1].child[-1]] + pnode.child[pos].child
                pnode.child[pos-1].child.pop()
            print(f'pnode : {pnode}, pnode.child : {pnode.child}')
            return True
        else:
            return False
            # pnode.child[pos].keys = pnode.child[pos-1].key[-1] + pnode.child[pos].keys
    def merge(self,pnode,ppos,lpos,rpos):
        print("MERGE ##-##- pnode : ",pnode, "ppos: ",ppos,"\t l,r pos : ",lpos,rpos)
        print("MergingChild"," lpos : " , pnode.child[lpos]," Ppos : " , pnode.keys[ppos]," rpos : " , pnode.child[rpos])
        pnode.child[lpos].keys = pnode.child[lpos].keys + [pnode.keys[ppos]] + pnode.child[rpos].keys
        pnode.keys.pop(ppos)
        if pnode.child[lpos].child:
            print("CHILD MERGING이 필요해",pnode.child[lpos].child, "<->",pnode.child[rpos].child)
            pnode.child[lpos].child += pnode.child[rpos].child
        pnode.child.pop(rpos)
            # pass
        pass


bt = BTree(5)
for i in range(1,50):
    bt.insert_node(bt.root, [i*10,i*10])
for i in range(1,25):
    bt.delete_pack(bt.root,[i*10,22])


DELETE Node :[[90, 90], [180, 180], [270, 270], [360, 360]] KEY :  [10, 22] 0
DELETE Node :[[30, 30], [60, 60]] KEY :  [10, 22] 0
DELETE Node :[[10, 10], [20, 20]] KEY :  [10, 22] 0
find node :  Node :[[20, 20]]  key :  [10, 22]
Check The Node Node :[[30, 30], [60, 60]] 's Stability
Not Stable => Balancing
Balancing Child  Node :[[30, 30], [60, 60]] cpos : 0 Node :[[20, 20]]
맨 왼쪽 자식 rebalancing
Merge
MERGE ##-##- pnode :  Node :[[30, 30], [60, 60]] ppos:  0 	 l,r pos :  0 1
MergingChild  lpos :  Node :[[20, 20]]  Ppos :  [30, 30]  rpos :  Node :[[40, 40], [50, 50]]
Check The Node Node :[[90, 90], [180, 180], [270, 270], [360, 360]] 's Stability
Not Stable => Balancing
Balancing Child  Node :[[90, 90], [180, 180], [270, 270], [360, 360]] cpos : 0 Node :[[60, 60]]
맨 왼쪽 자식 rebalancing
Merge
MERGE ##-##- pnode :  Node :[[90, 90], [180, 180], [270, 270], [360, 360]] ppos:  0 	 l,r pos :  0 1
MergingChild  lpos :  Node :[[60, 60]]  Ppos :  [90, 90]  rpos :  Node :[[120, 120], [150, 150]]
CHI

In [482]:
# bt.search_key(bt.root, [290,None])

(Node :[[280, 280], [290, 290]], [290, 290])

In [476]:
bt.print_inorder(bt.root)


[250, 250]
[260, 260]
[270, 270]
[280, 280]
[290, 290]
[300, 300]
[310, 310]
[320, 320]
[330, 330]
[340, 340]
[350, 350]
[360, 360]
[370, 370]
[380, 380]
[390, 390]
[400, 400]
[410, 410]
[420, 420]
[430, 430]
[440, 440]
[450, 450]
[460, 460]
[470, 470]
[480, 480]
[490, 490]


In [132]:
# bt.search_key(bt.root,[1,10])
# bt.print_inorder(bt.root)
print(bt.root.child[0] , bt.root.child[1])
print()
bt.root.child[0].child , bt.root.child[1].child

k:[[40, 40]] k:[[120, 120], [160, 160]]



([k:[[30, 30]], k:[[60, 60]]],
 [k:[[100, 100]], k:[[140, 140]], k:[[180, 180]]])

In [8]:
import csv
def compare(file1,file2):
    t,f=0,0
    with open(file1,'r') as f1:
        with open(file2,'r') as f2:
            reader1=csv.reader(f1, delimiter='\t')
            reader2=csv.reader(f2, delimiter='\t')
            for l1,l2 in zip(reader1,reader2):
                # print(l1,l2)
                if l1==l2:
                    t+=1
                else:
                    f+=1
    return t,f
compare("./data/input.csv", './data/input_compare.csv')

(1000000, 0)

In [35]:
def compare_df(origin,compare):
    odf=pd.read_csv(origin, sep="\t", names=["key","value"])
    cdf=pd.read_csv(compare, sep=",", names=["key","value"])
    mdf=pd.merge(odf,cdf,how="outer",on='key', suffixes=["_origin","_compare"]).fillna("NF")
    mdf["incorrect"]=mdf.apply(lambda x : 0 if x['value_origin']==x['value_compare'] else 1, axis="columns")
    return mdf

mdf = compare_df("./data/input3.csv", "./data/input3_compare.csv")
mdf

,key,value_origin,value_compare,incorrect
0,1,1,1,0
1,2,2,2,0
2,3,3,3,0
3,4,4,4,0
4,5,5,5,0
5,6,6,6,0
6,7,7,7,0
7,8,8,8,0
8,9,9,9,0
9,10,10,10,0


In [39]:
len(mdf), sum(mdf['incorrect'])

(15, 0)

In [42]:
import pandas as pd
df=pd.read_csv("./data/input3_compare.csv", sep="\t", names=["key","value"])
# df.to_csv("./data/input3_compare.csv", index=False, sep=",",encoding='utf-8',header=False, mode='w')
df

,key,value


In [47]:
import pandas as pd
from tqdm import tqdm
import os

def search(dir): return os.listdir(dir)
def compare_df(origin,compare):
    odf=pd.read_csv(origin, sep="\t", names=["key","value"])
    cdf=pd.read_csv(compare, sep=",", names=["key","value"])
    mdf=pd.merge(odf,cdf,how="outer",on='key', suffixes=["_origin","_compare"]).fillna("NF")
    mdf["incorrect"]=mdf.apply(lambda x : 0 if x['value_origin']==x['value_compare'] else 1, axis="columns")
    return mdf

tqdm.pandas()

bt=BTree(5)
# num = input("1. insertion \t2.deletion \t3.quit \n")
# if num=='1': # Insertion
## Insert
num = 1
fnames=', '.join([i.split('.')[0] for i in search('./data') if i[:5]=='input'])
inputname=input(f"Write the Input file name With out extension \n(ex. {fnames} )\n")
fname="./data/" + inputname + '.csv'
compare_name = f"./data/{inputname}_compare.csv"
print("\nInsertion Start\n")
inputdf=pd.read_csv(fname,sep='\t',names=['key','value'])
for idx,row in tqdm(inputdf.iterrows(), total=inputdf.shape[0]):
    bt.insert_node(bt.root, [row['key'],row['value']])

print("\nSearch Start\n")
# Search and Write to CSV            
arr=[]
for idx,row in tqdm(inputdf.iterrows(), total=inputdf.shape[0]):
        _ , kv = bt.search_key(bt.root, [row['key'],None])
        arr.append(kv)
sdf = pd.DataFrame(arr, columns=['key','value'])
sdf.to_csv(compare_name, index=False, sep=",",encoding='utf-8',header=False, mode='w')

## Compare
print("\nCompare Start\n")
mdf=compare_df(fname, f'./data/{inputname}_compare.csv')
t , f = len(mdf)-sum(mdf['incorrect']) , sum(mdf['incorrect'])
print(f'Data Count = {t+f} , Correct : {t} , Incorrect : {f} , True Percent : {100*t/(t+f)}%')





Insertion Start



100%|██████████| 15/15 [00:00<00:00, 21276.48it/s]



Search Start



100%|██████████| 15/15 [00:00<00:00, 16358.44it/s]


Compare Start

Data Count = 15 , Correct : 15 , Incorrect : 0 , True Percent : 100.0%


In [48]:
sdf

,key,value
0,1,1
1,2,2
2,3,3
3,4,4
4,5,5
5,6,6
6,7,7
7,8,8
8,9,9
9,10,10
